# Access the Crystallographic Open Database from QCrBox
The COD is a database of crystal structures determined by single crystal XRD that is fully available to the public

## Import and setting up folders / paths

In [7]:
import shutil

from qcrbox_wrapper import QCrBoxWrapper, QCrBoxPathHelper


We create a path to run our COD examples and get our paths from our own computer's filesystem and the internal filesystem of the QCrBox containers.

Using the dotenv package makes things more convenient as we can read the environment variables from the .env.dev file in the QCrBox directory. If you want to run with python core packages only, use the `__init__` method instead by defining the path to the shared directory explicitely in `path_to_shared_dir` and replacing the next four lines with:

```python
pathhelper = QCrBoxPathHelper(
    path_to_shared_dir,
    'examples_cod'
)
```

In [8]:
pathhelper = QCrBoxPathHelper.from_dotenv(
    '.env.dev',
    'examples_cod'
)

We now have two python pathlib objects we can use to make the reference to our pathes much easier. We have also created an examples_cod folder we will use to work for this notebook.

In [9]:
path_local = pathhelper.local_path
path_qcrbox = pathhelper.qcrbox_path

Let us first connect to the QCrBox. Make sure that you have actually started up the container with `qcb up` in the terminal. Sometimes it can take a while for the Server to come online, so you might need to retry a few times. 

In [10]:
qcrbox = QCrBoxWrapper.from_server_addr('127.0.0.1', 11000)

We can now see what applications are currently available and how we can access them. COD Check should show up here and therefore we select it in the new cell.

In [11]:
qcrbox.application_dict

2024-08-22T13:24:23.963735Z [debug    ] TODO: implement proper construction and validation of gui_url extra={}


{'COD Check': <COD Check>}

In [12]:
cod = qcrbox.application_dict['COD Check']

2024-08-22T13:24:28.238087Z [debug    ] TODO: implement proper construction and validation of gui_url extra={}


Using the python buildin help function we can see which commands are available for the given container.

In [13]:
help(cod)

Help on QCrBoxApplication in module qcrbox_wrapper.qcrbox_application object:

class QCrBoxApplication(builtins.object)
 |  QCrBoxApplication(application_spec: pyqcrbox.sql_models_NEW_v2.application_spec.ApplicationSpecWithCommands, wrapper_parent: 'QCrBoxWrapper') -> None
 |
 |  Represents an application in QCrBox packaged in its own container.
 |
 |  Methods defined here:
 |
 |  __init__(self, application_spec: pyqcrbox.sql_models_NEW_v2.application_spec.ApplicationSpecWithCommands, wrapper_parent: 'QCrBoxWrapper') -> None
 |      Initializes the QCrBoxApplication instance.
 |
 |      Parameters
 |      ----------
 |      application_spec: sql_models_NEW_v2.ApplicationSpecWithCommands
 |          The application spec as returned by the API endpoint `/applications`.
 |
 |  __repr__(self) -> str
 |      Return repr(self).
 |
 |  interactive_session(self, **kwargs)
 |
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |
 |  

We can now run our first command. QCrBox works with a subsect of cif entries, which will be called *unified cif entries* in this explanation. The subset are the base cif entries in the current cif dictionaries (as of March 2024). The conversion of these entries into a format the individual programs can understand is programmed into the commands themselves. This way the aliases of the cif format do not need to be taken into account by the developers of the software. If you want to create such a cif file from any cif, look at the `qcrboxtools.to_unified_cif function` or the QCrBoxTools library.

# Getting the number of entries with similar cell parameters and content in the COD

For the time being we copy our example cif file into the folder we will work in. We then use the `cod.get_number_fitting_cod_entries` to get the number of similar entries written to the file `nentries.json` in our work directory. Note that the path passed to the command is relative to the container file system (derived from `path_qcrbox`).

Finally, all functions in containers are executed asynchronously and non-blocking in our script. In order to wait until the function has been executed we can use the `wait_while_running` function of the returned calculated object

In [14]:
folder_test = path_local / 'example_count'
folder_test.mkdir(exist_ok=True)

shutil.copy('./input_files/cap_result.cif', folder_test / 'input.cif')

calc= cod.get_number_fitting_cod_entries(
    input_cif_path=path_qcrbox / 'example_count' / 'input.cif',
    cellpar_deviation_perc=5.0,
    listed_elements_only=True
)

calc.wait_while_running(0.5)

In [15]:
calc.status

<CalculationStatusEnum.SUCCESSFUL: 'successful'>

# Copying the atomic positions from the closest match in the COD

Here we download the structure with the closest cell parameters, given that the elements are the same and insert the cell parameters into an existing cif file.

In [16]:
folder_test2 = path_local / 'example_merge'
folder_test2.mkdir(exist_ok=True)

shutil.copy('./input_files/cap_result.cif', folder_test2 / 'input.cif')

calc= cod.merge_closest_cod_entry(
    input_cif_path=path_qcrbox / 'example_merge' / 'input.cif',
    output_cif_path=path_qcrbox / 'example_merge' / 'output.cif',
    cellpar_deviation_perc=5.0,
    listed_elements_only=True
)

In [18]:
calc.status

<CalculationStatusEnum.SUCCESSFUL: 'successful'>